In [1]:
!nvidia-smi

Mon Nov 27 05:28:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install --upgrade  textblob gensim pytorch-nlp swifter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16505 sha256=aa78f9ab7f35642a006db8ee72b411e49822ad722de8408804df088a7fc28213
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [3]:
import multiprocessing
import sys

import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob

embedding_dim = 100
epochs=10
batch_size = 250
corpus_size=5000

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  random.seed(42)


set_seeds_and_trace()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi

Writing get_data.sh


In [5]:
!bash get_data.sh

--2023-11-27 05:29:03--  https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.67.18, 2620:100:6031:18::a27d:5112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.67.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/352x7xzivf60zgc/news.csv [following]
--2023-11-27 05:29:04--  https://www.dropbox.com/s/raw/352x7xzivf60zgc/news.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca9eb10eefe777cc2db81fad3d4.dl.dropboxusercontent.com/cd/0/inline/CIV1UkdUpFdcLKmxP7gMeUCO_sKkqgsk6yJ56A0nHOgdzAqlAXQayH42TGLcyaavkzpMCm_mcDQNWxF0Jfv0kG8M5B1sze0839ScNS_YSS8HizG3s5DyQPbfGenkejHj7E2_urnU4cl3y2lNIQn9RHAc/file# [following]
--2023-11-27 05:29:05--  https://uca9eb10eefe777cc2db81fad3d4.dl.dropboxusercontent.com/cd/0/inline/CIV1UkdUpFdcLKmxP7gMeUCO_sKkqgsk6yJ56A0nHOgdzAqlAXQayH42TGLcyaavkzpMCm_mcDQNWxF0Jfv0kG8M5B1sze0839ScNS_YSS

In [6]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

In [7]:
news_pre

,category,title
0,Business,"BBC set for major shake-up, claims newspaper"
1,Business,Marsh averts cash crunch
2,Sports,"Jeter, Yankees Look to Take Control (AP)"
3,Sci/Tech,Flying the Sun to Safety
4,Business,Stocks Seen Flat as Nortel and Oil Weigh
...,...,...
4995,Sci/Tech,News: Technology Already Exists to Stabilize G...
4996,Sci/Tech,Telecoms Tie-Up
4997,Sci/Tech,Justice OKs Cingular-AT T Wireless Deal (AP)
4998,World,Zimbabwe Denies Reports of Food Shortage (AP)


In [8]:
import re
def preprocess_text(text, should_join=True):
    # Use the tokenizer to tokenize into words, lowercase them, remove punctuation, and finally use gensim.utils.simple_preprocess(text)
    text = ' '.join(gensim.utils.tokenize(text, lowercase=True))
    text = re.sub(r"[.,!?]", r" ", text)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [9]:
preprocess_text('This is the best night of my life! Is it? Well, maybe')

'this is the best night of my life is it well maybe'

In [10]:
import swifter
news = news_pre.title.swifter.apply(preprocess_text)

Pandas Apply:   0%|          | 0/5000 [00:00<?, ?it/s]

In [11]:
news.to_csv('news_pre.csv', header=False, index=False)

In [12]:
!head -n 5 news_pre.csv

bbc set for major shake up claims newspaper
marsh averts cash crunch
jeter yankees look to take control ap
flying the sun to safety
stocks seen flat as nortel and oil weigh


In [13]:

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_pre.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)

from gensim.models import Word2Vec
word2vec_model = Word2Vec(sentences=MyCorpus(), vector_size=embedding_dim)
# Get a word2vec model using gensim.models and passing the sentences using MyCorpus()

In [14]:
wv_model = word2vec_model.wv

In [16]:
weights = torch.Tensor(wv_model.vectors)  # Get the weights of the model (the embedding) and convert to tensor. Hint: Check word2vec_model.wv
vocab_size = len(wv_model.index_to_key)  # get vocab size from index_to_key in word2vec_model.wv

In [17]:
weights.shape

torch.Size([1430, 100])

In [18]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news
news_preprocessed

,label,title
0,0,bbc set for major shake up claims newspaper
1,0,marsh averts cash crunch
2,1,jeter yankees look to take control ap
3,2,flying the sun to safety
4,0,stocks seen flat as nortel and oil weigh
...,...,...
4995,2,news technology already exists to stabilize gl...
4996,2,telecoms tie up
4997,2,justice oks cingular at wireless deal ap
4998,3,zimbabwe denies reports of food shortage ap


In [19]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)   # Since 2 titles may have different number of words, we have to find the max length and fill with 0s if a title is shorter

In [20]:
maximum

17

In [21]:
X = np.zeros((len(news_preprocessed), maximum))   # Here we do what we said above
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# HINT: to iterate through a column of a pandas dataframe you do:

# for index, value in df.iterrows():
#      #do something

for index, row in news_preprocessed.iterrows():
  word_ix = 0
  for word in textblob_tokenizer(row.title):
    token = vocab_size+1
    if word in wv_model.key_to_index:
      token = wv_model.key_to_index[word]
    word_ix += 1

# FILL
y = news_preprocessed.label

In [22]:
X[:2]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]])

In [32]:
import torch.nn.functional as F

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.Tensor(X_train).to(torch.long)
X_test = torch.Tensor(X_test).to(torch.long)
# Convert y_train and y_test from an array of values between 0-3 to a one hot matrix tensor
y_train = F.one_hot(torch.Tensor(y_train.to_numpy()).to(torch.long))
y_test = F.one_hot(torch.Tensor(y_test.to_numpy()).to(torch.long))


In [33]:
class MeanLayer(nn.Module):

  def forward(self, x):
    return torch.mean(x, dim=1)

In [62]:
model = nn.Sequential(
    nn.Embedding(vocab_size, embedding_dim),
    nn.Linear(embedding_dim, 100),
    nn.ReLU(),
    nn.Linear(100, 50),
    nn.ReLU(),
    MeanLayer(),
    nn.Linear(50, 4),
    nn.Softmax()
)

In [55]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [60]:
def train_cbow(X, y, model, loss_function, optimizer, epochs):
    for epoch in range(epochs):
        total_loss = 0

        # Step 1. Recall that torch *accumulates* gradients. Before passing in a new instance,
        # you need to zero out the gradients from the old instance
        optimizer.zero_grad()

        # Step 2. Run the forward pass, getting log probabilities over next words
        log_probs = model(X)
        # Step 3. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, y.to(torch.float))

        # Step 4. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Print progress
        if (epoch + 1) % 10 == 0:
            print('Epoch: {}, Loss: {:.4f}'.format(epoch + 1, total_loss))
    return model

In [61]:
trained_model = train_cbow(X_train, y_train, model, loss_function, optimizer, epochs=epochs)

Epoch: 10, Loss: 1.3880


## Exercise extra-credit: Make X and y a DataLoader, add batching, and validate the performance with the test set